In [6]:
import math
import numpy as np

# Project 2: **Harmonic Vibrational Analysis**


This project demonstrates a normal coordinate/harmonic vibrational frequency calculation.

### Step 1: **Read the Coordinate Data**

The coordinate data are given in a format identical to that for Project 1. The test case for the remainder of this project is the water molecule, optimized at the SCF/DZP level of theory.

In [7]:
geom2=np.genfromtxt("h2o_geom.txt", skip_header=1)

### Step 2: **Read the Cartesian Hessian Data**

The primary input data for the harmonic vibrational calculation is the Hessian matrix, which consists of second derivatives of the energy with respect to atomic positions $$F_{ij}=\frac{\partial^2V}{\partial q_i\partial q_j}$$
$$$$
The first integer in the file is the number of atoms (which you should compare to the corresponding value from the geometry file as a test of consistency), while the remaining values have the following format $$\begin{matrix} F_{x_1,x_1} & F_{x_1,y_1} & F_{x_1,z_1} \\ F_{x_1,x_2} & F_{x_1,y_2} & F_{x_1,z_2} \\ F_{x_1,x_3} & F_{x_1,y_3} & F_{x_1,z_3} \\ F_{y_1,x_1} & F_{y_1,y_1} & F_{y_1,z_1} \\ F_{y_1,x_2} & F_{y_1,y_2} & F_{y_1,z_2} \\ \vdots & \vdots & \vdots \\ F_{z_3,x_3} & F_{z_3,y_3} & F_{z_3,z_3} \end{matrix}$$

While the format of the input file is rectangular, the Hessian should be rearranged in a square matrix $$\begin{matrix}F_{x_1,x_1} & F_{x_1,y_1} & F_{x_1,z_1} & F_{x_1,x_2} & F_{x_1,y_2} & F_{x_1,z_2} & F_{x_1,x_3} & F_{x_1,y_3} & F_{x_1,z_3} \\ F_{y_1,x_1} & F_{y_1,y_1} & F_{y_1,z_1} & F_{y_1,x_2} & F_{y_1,y_2} & F_{y_1,z_2} & F_{y_1,x_3} & F_{y_1,y_3} & F_{y_1,z_3} \\ \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots \ \ \\ F_{z_3,x_3} & F_{z_3,y_3} & F_{z_3,z_3} & F_{z_3,x_3} & F_{z_3,y_3} & F_{z_3,z_3} & F_{z_3,x_3} & F_{z_3,y_3} & F_{z_3,z_3}\end{matrix}$$

In [8]:
hess2=np.genfromtxt("h2o_hessian.txt", skip_header=1)

def making_square_hess(hess,geom):
  H = np.zeros([3*len(geom),3*len(geom)])
  for i in range(3*len(geom)):
    for j in range(len(geom)):
      for k in range(len(geom)):
        H[i,(k+j*3)]=hess[(i*3+j),k]
  return H

F = making_square_hess(hess2,geom2)

### Step 3: **Mass-Weight the Hessian Matrix**

Divide each element of the Hessian matrix by the product of square-roots of the masses of the atoms associated with the given coordinates $$F_{ij}^M=\frac{F_{ij}}{\sqrt{m_im_j}}$$ where $m_i$ represents the mass of the atom corresponding to atom $i$

In [9]:
masses = np.array([0.,1.00782503207,4.00260325415,7.016004548,9.012182201,11.009305406, 12,14.00307400478,15.99491461956,18.998403224,19.99244017542,
22.98976928087,23.985041699,26.981538627,27.97692653246,30.973761629,31.972070999,34.968852682,39.96238312251,38.963706679,39.962590983,44.955911909,
47.947946281,50.943959507,51.940507472,54.938045141,55.934937475,58.933195048,57.935342907,62.929597474,63.929142222,68.925573587,73.921177767,
74.921596478,79.916521271,78.918337087,85.910610729,84.911789737,87.905612124,88.905848295,89.904704416,92.906378058,97.905408169,98.906254747,
101.904349312,102.905504292,105.903485715,106.90509682,113.90335854,114.903878484,119.902194676,120.903815686,129.906224399,126.904472681,131.904153457,
132.905451932,137.905247237,138.906353267,139.905438706,140.907652769,141.907723297,144.912749023,151.919732425,152.921230339,157.924103912,158.925346757,
163.929174751,164.93032207,165.930293061,168.93421325,173.938862089,174.940771819,179.946549953,180.947995763,183.950931188,186.955753109,191.96148069,
192.96292643,194.964791134,196.966568662,201.970643011,204.974427541,207.976652071,208.980398734,208.982430435,210.987496271,222.017577738,222.01755173,
228.031070292,227.027752127,232.038055325,231.03588399,238.050788247,237.048173444,242.058742611,243.06138108,247.07035354,247.07030708,251.079586788,
252.082978512,257.095104724,258.098431319,255.093241131,260.105504,263.112547,255.107398,259.114500,262.122892,263.128558,265.136151,281.162061,272.153615,
283.171792,283.176451,285.183698,287.191186,292.199786,291.206564,293.214670]) #data taken from https://github.com/psi4/psi3/blob/master/include/masses.h 😏

def add_mass_column(geom): # to dd a column to the geom, for the mass of the atoms
  geom = np.c_[geom, np.zeros(len(geom))]  # add an all-zero column to our geom array
  for i in range(len(geom)): 
    geom[i,4] = masses[int(geom[i,0])]
  return geom

geom2=add_mass_column(geom2)

def making_mass_hess(square_hess,geom):
  mass_arr = np.zeros([3*len(geom)])
  for i in range(len(geom)):
    for j in range(3):
      mass_arr[i*3+j]=geom[i,4]
  mass_hess = np.zeros([3*len(geom),3*len(geom)])
  for i in range(3*len(geom)):
    for j in range(3*len(geom)):
      mass_hess[i,j]=square_hess[i,j]/(math.sqrt(mass_arr[i]*mass_arr[j]))
  return mass_hess

F_M = making_mass_hess(F,geom2)

## print test comment
print("\n For the H\u2082O test case, the 9\u00d79 mass-weighted Hessian is \n")
for row in F_M:
  print(' '.join(map(lambda x: "{:11.7f}".format(x), row)))
print("\n",end="")


 For the H₂O test case, the 9×9 mass-weighted Hessian is 

  0.0057996   0.0000000   0.0000000  -0.0115523   0.0000000   0.0000000  -0.0115523   0.0000000   0.0000000
  0.0000000   0.0198271   0.0000000   0.0000000  -0.0394937   0.0199304   0.0000000  -0.0394937  -0.0199304
  0.0000000   0.0000000   0.0175112   0.0000000   0.0086617  -0.0348807   0.0000000  -0.0086617  -0.0348807
 -0.0115523   0.0000000   0.0000000   0.0510672   0.0000000   0.0000000  -0.0050452   0.0000000   0.0000000
  0.0000000  -0.0394937   0.0086617   0.0000000   0.1716643  -0.0569527   0.0000000  -0.0143291   0.0224462
  0.0000000   0.0199304  -0.0348807   0.0000000  -0.0569527   0.1258525   0.0000000  -0.0224462   0.0131055
 -0.0115523   0.0000000   0.0000000  -0.0050452   0.0000000   0.0000000   0.0510672   0.0000000   0.0000000
  0.0000000  -0.0394937  -0.0086617   0.0000000  -0.0143291  -0.0224462   0.0000000   0.1716643   0.0569527
  0.0000000  -0.0199304  -0.0348807   0.0000000   0.0224462   0.0131055   0.

### Step 4: **Eigenvalues of the Mass-Weighted Hessian Matrix**

Compute the eigenvalues of the mass-weighted Hessian.



In [10]:
F_M_eigval, F_M_eigvec = np.linalg.eig(F_M)
F_M_eigval = np.sort(F_M_eigval)[::-1] # sort the eigenvalues in descending order

print("\n The eigenvalues of the mass-weighted Hessian for the H\u2082O test case (in atomic units) are \n")
np.savetxt(sys.stdout, F_M_eigval, fmt="%18.10f")  # this is to print the values vertically, without brackets
print("\n",end="")


 The eigenvalues of the mass-weighted Hessian for the H₂O test case (in atomic units) are 

      0.2351542374
      0.2107113147
      0.1317512795
      0.0561123956
      0.0547551460
      0.0518216600
      0.0000000000
     -0.0000000000
     -0.0000000000



### Step 5: **Compute the Harmonic Vibrational Frequencies**

The vibrational frequencies are proportional to the squareroot of the eigenvalues of the mass-weighted Hessian $$\omega_i=constant\sqrt{λ_i}$$

In [11]:
F_M_eigval = F_M_eigval/5.29177249e-9  # convert from bohr-1 to cm-1 ?

omega = np.sqrt(np.absolute(F_M_eigval)) # I don't know the value of the constant 🤷‍♂️, so i just did the square-root 😕

print("\n The harmonic vibrational frequencies for the H\u2082O test case (in cm\u207b\u00b9) are \n")
np.savetxt(sys.stdout, omega, fmt="%12.4f")  # this is to print the values vertically, without brackets
print("\n",end="")


 The harmonic vibrational frequencies for the H₂O test case (in cm⁻¹) are 

   6666.1614
   6310.2032
   4989.7276
   3256.3331
   3216.7098
   3129.3568
      0.0561
      0.0000
      0.0000



Note that there are only three zero frequencies in this case when there should be six. This is because the structure used in the computation is not a stationary point on the potential energy surface, and thus the three “rotational frequencies” are non-zero.